# Causal effect estimation with external controls

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import clone

import warnings
from tqdm import trange

In [12]:
%run "../scripts/auxilary.py"
%run "../scripts/dml_ate_fusion.py"
%run "../scripts/simu.py"

We consider a simulation setting where we have a relatively small (n = 200) target experimental data, with two large (n = 1000) source data. This is a common setting in practice. Source data are heterogeneous with both covariate shift and label shift. However, the conditional outcome on the control is the same as the target data. 

In [14]:
# Global configuration for the simulation
# nT: sample size of target data
# nS_1: sample size of source data 1
# nS_2: sample size of source data 2
# n_reps: number of simulation repetitions
# seed: random seed for reproducibility
CONFIG = {
    "nT": 200,
    "nS_1": 1000,
    "nS_2": 1000,
    "n_reps": 1000, 
    "seed": 20250901
}


Our estimator requires three ML models: a mean model, a propensity score model, and a variance model. Here we consider random forest as an example, but other advanced ML models such as XGBoost, LightGBM, and neural networks, are also readily incorporated. 

In addition to these ML models, we also require an estimator of the density ratio. Here we use kernel density estimation, but other methods such as logistic regression or entropy balcancing, are also available for use.

In [15]:
# Define the models to be used for nuisance function estimation, here we use random forest as an example
mean_model = RandomForestRegressor(n_estimators=100)
p_score_model = RandomForestRegressor(n_estimators=100)
var_model = RandomForestRegressor(n_estimators=100)

In [16]:
# Exchangeable functional
mu_0 = lambda U1, U2: 1 + 0.5*U1 + U2

# Functions to generate the target data (The true average treatment effect is 2)
target_p_score = lambda U1, U2: 1/(1+np.exp(-0.2*U1-0.3*U2-np.sign(U2)*1.5*U1**2))
target_mu_1 = lambda U1, U2: 3 + 0.5*U1 + 1.5*U2
target_sigma_0 = lambda U1, U2: 0.2 * np.abs(U1)**0.4
target_sigma_1 = lambda U1, U2: 0.2 * np.abs(U2)**0.2

# Functions to generate the 1st source data (The average treatment effect is 0)
source_1_p_score = lambda U1, U2: 1/(1+np.exp(-0.2*U1-0.3*U2))
source_1_mu_1 = lambda U1, U2:  1+ 0.5*U2 + 1.5*U1
source_1_sigma_0 = lambda U1, U2: 0.2 * np.abs(U1)**0.2
source_1_sigma_1 = lambda U1, U2: 0.2 * np.abs(U2)**0.2

# Functions to generate the 2nd source data (The average treatment effect is -1)
source_2_p_score = lambda U1, U2: 1/(1+np.exp(-0.2*U1-0.3*U2))
source_2_mu_1 = lambda U1, U2:  -U1 + U2
source_2_sigma_0 = lambda U1, U2: 0.2 * np.abs(U1)**0.4
source_2_sigma_1 = lambda U1, U2: 0.2 * np.abs(U2)**0.4

# Except the exchangeable functional, all other functions are heterogeneous between target and source data.

In [17]:
def MainSimu(nT, nS, seed, n_reps,
             mu_0, target_p_score, target_mu_1, target_sigma_0, target_sigma_1, 
             source_p_score, source_mu_1, source_sigma_0, source_sigma_1, 
             mean_mode, p_score_model, var_model):
    """
    Main simulation function to run DML_ATE multiple times and summarize the results. 

    Parameters
    ----------
    n : int, number of samples in each simulation
    seed : int, random seed
    n_reps : int, number of repetitions
    mu_0 : function of U1, U2, mean function of Y when A=0 on both target and source data
    target_p_score : function of U1, U2, propensity score function on target data
    target_mu_1 : function of U1, U2, mean function of Y when A=1 on target data
    target_sigma_0 : function of U1, U2, variance function of Y when A=0 on target data
    target_sigma_1 : function of U1, U2, variance deviation function of Y when A=1 on target data
    source_p_score : list of functions of U1, U2, propensity score function on each source data
    source_mu_1 : list of functions of U1, U2, mean function of Y when A=1 on each source data
    source_sigma_0 : list of functions of U1, U2, variance function of Y when A=0 on each source data
    source_sigma_1 : list of functions of U1, U2, variance deviation function of Y when A=1 on each source data
    mean_model: machine learning model class, estimate the conditional mean given covariates
    p_score_model: machine learning model class, estimate the propensity score given covariates
    var_model: machine learning model class, estimate the conditional variance given covariates

    Returns
    -------
    pd.DataFrame with columns "Coverage", "Bias", "Std_Error", DML ATE with target data only
    pd.DataFrame with columns "Coverage", "Bias", "Std_Error", DML ATE with all data

    """
    # True ATE
    true_ATE = 2.0
    coverages_fusion = []
    biases_fusion = []
    std_errors_fusion = []
    coverages = []
    biases = []
    std_errors = []
    for rep in trange(n_reps, desc="Simulations"):
        rep_seed = seed + rep
        np.random.seed(rep_seed)
        
        # Generate target data and estimate ATE using target data only
        target_data = SimuData(nT, target_p_score, mu_0, target_mu_1, target_sigma_0, target_sigma_1)
        ATE_est, ATE_std = DML_ATE(target_data, p_score_model, mean_model)

        # Generate source data and estimate ATE using all data
        source_data = [SimuData(nS, source_p_score[m], mu_0, source_mu_1[m], source_sigma_0[m], source_sigma_1[m]) for m in range(len(source_p_score))]
        ATE_est_fusion, ATE_std_fusion = DML_ATE_fusion(target_data, source_data, p_score_model, mean_model, var_model, density_ratio)
        coverage_fusion = (ATE_est_fusion - 1.96 * ATE_std_fusion <= true_ATE) and (ATE_est_fusion + 1.96 * ATE_std_fusion >= true_ATE)
        bias_fusion = np.abs(ATE_est_fusion - true_ATE)
        std_error_fusion = ATE_std_fusion
        coverages_fusion.append(coverage_fusion)
        biases_fusion.append(bias_fusion)
        std_errors_fusion.append(std_error_fusion)

        coverage = (ATE_est - 1.96 * ATE_std <= true_ATE) and (ATE_est + 1.96 * ATE_std >= true_ATE)
        bias = np.abs(ATE_est - true_ATE)
        std_error = ATE_std
        coverages.append(coverage)
        biases.append(bias)
        std_errors.append(std_error)
    result = pd.DataFrame({"Coverage": coverages, "Bias": biases, "Std_Error": std_errors})
    result_fusion = pd.DataFrame({"Coverage": coverages_fusion, "Bias": biases_fusion, "Std_Error": std_errors_fusion}) 
    return result, result_fusion

Simulation is repeated for 1,000 times, and the coverage and standard error of two estimators, the classical DML estimator for ATE and our data fusion estimator, are compared.

In [65]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    result, result_fusion = MainSimu(CONFIG["nT"], CONFIG["nS_1"], CONFIG["seed"], CONFIG["n_reps"],
                                     mu_0, target_p_score, target_mu_1, target_sigma_0, target_sigma_1, 
                                     [source_1_p_score, source_2_p_score], [source_1_mu_1, source_2_mu_1], 
                                     [source_1_sigma_0, source_2_sigma_0], [source_1_sigma_1, source_2_sigma_1], 
                                     mean_model, p_score_model, var_model)

Simulations: 100%|██████████| 1000/1000 [1:31:51<00:00,  5.51s/it]


In [67]:
print(result["Coverage"].mean(), result["Bias"].mean(), result["Std_Error"].mean())
print(result_fusion["Coverage"].mean(), result_fusion["Bias"].mean(), result_fusion["Std_Error"].mean())

0.905 0.817855910143713 0.67931832731549
0.948 0.5972526514731746 0.5865094149401825


Due to a small sample target data (n = 200), coverage of the classical DML estimator is slightly below 95% (coverage = 0.905). In contrast, our data fusion estimator has valid coverage (0.948), and leads to 13.7% reduction in standard error (or 34% increase in effective sample size). 

We note that augmentation is asymptotically equivalent to pooling all data to estimate the conditional mean on the control arm. Therefore, the following three versions of the data fusion algorithms are all equivalent: (i) estimate the conditional mean on the control with all data, no augmentation term (ii) estimate the conditional mean on the control with only the target data, with the augmentation term (iii) estimate the conditional mean on the control with all data, with the augmentation term. For finite-sample performance, (iii) is used.